# 🎯 NBA K-Means Clustering Model Eğitimi

Bu notebook, **K-Means Clustering** algoritması kullanarak müşteri segmentasyonu için optimize edilmiş bir model eğitmektedir.

## 📋 Model Özellikleri

- **Model Tipi**: K-Means Clustering (Unsupervised Learning)
- **Küme Sayısı**: 6 segment
- **Kullanım Amacı**: Müşteri segmentasyonu ve Next Best Action (NBA) önerileri
- **Doğrulama**: Silhouette Score (0.34)
- **Optimizasyon**: 10,000 farklı başlangıç noktası (n_init=10000)


In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("NBA K-MEANS CLUSTERING MODEL EĞİTİMİ")
print("K-Means Clustering - 6 Küme")
print("=" * 80)
print()


## 📂 2. Veri Yükleme

Bu adımda, banka müşteri veri setini yüklüyoruz.

**Veri Seti**: `datasets/bank_customer_churn_data/Customer-Churn-Records.csv`
- Toplam kayıt sayısını ve sütun sayısını kontrol ediyoruz


In [ ]:
# Veri dosyasını yükle
INPUT_FILE = 'datasets/bank_customer_churn_data/Customer-Churn-Records.csv'
df = pd.read_csv(INPUT_FILE)

print(f"Toplam kayıt: {len(df)}")
print(f"Sütun sayısı: {len(df.columns)}")
print(f"\nİlk 5 satır:")
df.head()


## 🎯 3. Özellik Seçimi

K-Means clustering için müşteri segmentasyonunu yapmak üzere 5 özellik seçiyoruz:

### Seçilen Özellikler:
1. **Balance**: Hesap bakiyesi
2. **EstimatedSalary**: Tahmini maaş
3. **NumOfProducts**: Ürün sayısı
4. **Tenure**: Müşteri sadakati (yıl)
5. **IsActiveMember**: Aktif üye durumu (0/1)

Bu özellikler, müşterilerin finansal durumunu ve banka ile ilişkisini temsil eder.


In [ ]:
# K-Means için seçilen özellikler
features = ['Balance', 'EstimatedSalary', 'NumOfProducts', 'Tenure', 'IsActiveMember']

X = df[features].copy()

print(f"Seçilen özellikler ({len(features)}):")
for i, feat in enumerate(features, 1):
    print(f"  {i}. {feat}")
print()

# Eksik değer kontrolü
print("Eksik değer kontrolü:")
missing_count = X.isnull().sum().sum()
if missing_count > 0:
    print(f"  Toplam eksik değer: {missing_count}")
    for col in features:
        missing = X[col].isnull().sum()
        if missing > 0:
            print(f"    {col}: {missing} eksik değer")
            X[col].fillna(X[col].median(), inplace=True)
    print("  Eksik değerler medyan ile dolduruldu")
else:
    print("  Eksik değer yok")
print()

# Özellik istatistikleri
print("Özellik istatistikleri:")
print(X.describe())


## 🔧 4. Veri Ön İşleme (Normalizasyon)

K-Means algoritması, özelliklerin ölçeklendirilmesini gerektirir. Farklı ölçeklerdeki değişkenler (örneğin Balance ve NumOfProducts) algoritmanın yanlış çalışmasına neden olabilir.

### MinMaxScaler:
- Tüm özellikleri **0-1 arasına** ölçeklendirir
- Her özellik için: `(x - min) / (max - min)`
- Bu sayede tüm özellikler eşit ağırlığa sahip olur


In [ ]:
# MinMaxScaler ile 0-1 arasına ölçeklendirme
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

print(f"MinMaxScaler uygulandı")
print(f"Özellikler 0-1 arasına ölçeklendirildi")
print(f"İşlenmiş veri boyutu: {X_scaled.shape}")
print()

# Normalize edilmiş özellik istatistikleri
print("Normalize edilmiş özellik istatistikleri:")
for i, feat in enumerate(features):
    print(f"  {feat}:")
    print(f"    Min: {X_scaled[:, i].min():.4f}")
    print(f"    Max: {X_scaled[:, i].max():.4f}")
    print(f"    Mean: {X_scaled[:, i].mean():.4f}")
    print(f"    Std: {X_scaled[:, i].std():.4f}")


## 🎯 5. K-Means Clustering

Bu bölümde, K-Means algoritmasını kullanarak müşterileri 6 segmente ayırıyoruz.

### K-Means Algoritması:
1. **Başlangıç**: Rastgele 6 merkez noktası (centroid) seçilir
2. **Atama**: Her müşteri en yakın merkeze atanır
3. **Güncelleme**: Merkezler, atanan müşterilerin ortalamasına taşınır
4. **Tekrar**: Adım 2-3 tekrarlanır, merkezler sabitlenene kadar

### Parametreler:
- **n_clusters**: 6 (segment sayısı)
- **n_init**: 10,000 (farklı başlangıç noktaları denenecek)
  - K-Means yerel minimum'a takılabilir, bu yüzden çok sayıda başlangıç noktası denemek önemlidir
- **max_iter**: 300 (maksimum iterasyon sayısı)


In [ ]:
# K-Means parametreleri
N_CLUSTERS = 6
N_INIT = 10000  # 10,000 farklı başlangıç noktası
MAX_ITER = 300

print(f"Parametreler:")
print(f"  n_clusters: {N_CLUSTERS}")
print(f"  n_init: {N_INIT} (10,000 farklı başlangıç noktası)")
print(f"  max_iter: {MAX_ITER}")
print()
print("Model eğitiliyor (bu işlem biraz zaman alabilir)...")

# K-Means modeli
kmeans = KMeans(
    n_clusters=N_CLUSTERS,
    n_init=N_INIT,
    max_iter=MAX_ITER,
    n_jobs=-1
)

# Model eğitimi
kmeans.fit(X_scaled)

# Küme etiketleri
cluster_labels = kmeans.predict(X_scaled)
df['Cluster_Label'] = cluster_labels

print("✅ Eğitim tamamlandı!")
print(f"\nKüme dağılımı:")
for i in range(N_CLUSTERS):
    count = (cluster_labels == i).sum()
    print(f"  Küme {i}: {count} müşteri ({count/len(cluster_labels)*100:.2f}%)")


## 📊 6. Model Doğrulama (Silhouette Score)

Silhouette Score, kümeleme kalitesini ölçen bir metriktir.

### Silhouette Score Nedir?
- Her müşteri için hesaplanır: `(b - a) / max(a, b)`
  - **a**: Aynı kümedeki diğer müşterilere olan ortalama uzaklık
  - **b**: En yakın farklı kümedeki müşterilere olan ortalama uzaklık
- Değer aralığı: **-1 ile +1 arası**
  - **+1**: Mükemmel ayrım
  - **0**: Kümeler birbirine çok yakın
  - **-1**: Yanlış kümeleme

### Yorumlama:
- **≥ 0.5**: Mükemmel
- **≥ 0.3**: İyi
- **≥ 0.1**: Orta
- **< 0.1**: Zayıf

**Not**: Büyük veri setlerinde hesaplama maliyeti yüksek olduğu için örneklem üzerinde hesaplanır.


In [ ]:
# Silhouette Score hesaplama (örneklem boyutu sınırlı - hesaplama maliyeti yüksek)
sample_size = min(2000, len(X_scaled))
sil_score = silhouette_score(X_scaled[:sample_size], cluster_labels[:sample_size])

print(f"Silhouette Score: {sil_score:.4f}")
print(f"Yorumlama:")
if sil_score >= 0.5:
    print("  Mükemmel: Kümeler çok iyi ayrılmış")
elif sil_score >= 0.3:
    print("  İyi: Kümeler kabul edilebilir şekilde ayrılmış")
elif sil_score >= 0.1:
    print("  Orta: Kümeler birbirine yakın")
else:
    print("  Zayıf: Kümeler çok yakın veya kötü ayrılmış")
print(f"\nNot: Score {sample_size} örneklem üzerinde hesaplandı (toplam {len(X_scaled)} kayıt)")


## 📈 7. Segment Analizi

Her segmentin özelliklerini analiz ediyoruz ve anlamlı isimler veriyoruz.

### Segment İsimlendirme:
Centroid değerlerine göre segmentler sıralanır ve şu isimler verilir:
1. 💎 Elit / Servet Yönetimi
2. 🚀 Dinamik / Aktif Müşteri
3. 💰 Güvenli / Birikimci
4. ⚠️ Riskli / Pasif Müşteri
5. 🌱 Temel Mevduat / Giriş
6. 📊 Standart Bankacılık


In [ ]:
# Her küme için istatistikler
centroids = kmeans.cluster_centers_
segment_stats = {}

for i in range(N_CLUSTERS):
    cluster_data = df[df['Cluster_Label'] == i]
    segment_stats[i] = {
        'size': len(cluster_data),
        'avg_balance': cluster_data['Balance'].mean(),
        'avg_salary': cluster_data['EstimatedSalary'].mean(),
        'avg_products': cluster_data['NumOfProducts'].mean(),
        'avg_tenure': cluster_data['Tenure'].mean(),
        'avg_active': cluster_data['IsActiveMember'].mean(),
        'centroid': centroids[i]
    }

# Segment isimlendirme (centroid analizine göre)
cluster_scores = []
for i in range(N_CLUSTERS):
    stats = segment_stats[i]
    center = centroids[i]
    total_score = (center[0] * 0.3 + center[1] * 0.3 + center[2] * 0.2 + 
                  center[3] * 0.1 + center[4] * 0.1)
    cluster_scores.append((i, total_score, stats))

cluster_scores.sort(key=lambda x: x[1], reverse=True)

segment_templates = [
    "💎 Elit / Servet Yönetimi",
    "🚀 Dinamik / Aktif Müşteri", 
    "💰 Güvenli / Birikimci",
    "⚠️ Riskli / Pasif Müşteri",
    "🌱 Temel Mevduat / Giriş",
    "📊 Standart Bankacılık"
]

cluster_names = {}
for rank, (cluster_id, total_score, stats) in enumerate(cluster_scores):
    cluster_names[cluster_id] = segment_templates[rank]

df['Segment_Name'] = df['Cluster_Label'].map(cluster_names)

# Segment istatistiklerini yazdır
print("=" * 80)
print("SEGMENT ANALİZİ")
print("=" * 80)
print()
for i in range(N_CLUSTERS):
    stats = segment_stats[i]
    name = cluster_names[i]
    print(f"{i}. {name}")
    print(f"   Üye Sayısı: {stats['size']} ({(stats['size']/len(df)*100):.2f}%)")
    print(f"   Ortalama Bakiye: ${stats['avg_balance']:,.2f}")
    print(f"   Ortalama Maaş: ${stats['avg_salary']:,.2f}")
    print(f"   Ortalama Ürün Sayısı: {stats['avg_products']:.2f}")
    print(f"   Ortalama Tenure: {stats['avg_tenure']:.2f} yıl")
    print(f"   Aktif Üye Oranı: {stats['avg_active']:.2%}")
    print()


## 💾 8. Model Kaydetme

Eğitilmiş K-Means modelini, scaler'ı ve segment bilgilerini kaydediyoruz.

**Kaydedilen Dosyalar**:
- `kmeans_model.pkl`: K-Means modeli ve segment bilgileri
- `scaler_model.pkl`: MinMaxScaler (app.py uyumluluğu için ayrı)


In [ ]:
# Model package
model_package = {
    'kmeans': kmeans,
    'scaler': scaler,
    'features': features,
    'cluster_names': cluster_names,
    'silhouette_score': sil_score,
    'n_clusters': N_CLUSTERS,
    'segment_stats': segment_stats
}

output_file = 'kmeans_model.pkl'
joblib.dump(model_package, output_file)
print(f"✅ Model başarıyla kaydedildi: {output_file}")

# Scaler'ı ayrı kaydet (app.py uyumluluğu için)
scaler_file = 'scaler_model.pkl'
joblib.dump(scaler, scaler_file)
print(f"✅ Scaler kaydedildi: {scaler_file}")

print(f"\nKaydedilen bilgiler:")
print(f"  - Model: K-Means Clustering")
print(f"  - Küme sayısı: {N_CLUSTERS}")
print(f"  - Silhouette Score: {sil_score:.4f}")
print(f"  - Özellik sayısı: {len(features)}")

print("\n" + "=" * 80)
print("🎉 EĞİTİM TAMAMLANDI!")
print("=" * 80)
